In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/39.OEMPartsOnline'
else:
    print('fuck')

path_root

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'./drive/MyDrive/code/03.crawler/39.OEMPartsOnline'

In [2]:
import pandas as pd

# = = =  = = =  = = =

import requests

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import json

# = = =  = = =  = = =

import itertools

# = = =  = = =  = = =

import re
import html as html_lib

In [3]:
directory_in = os.path.join(path_root, './file/2.url')
list_file = [file for file in sorted(list(os.walk(directory_in))[0][2]) if file not in ['.DS_Store'] and os.path.isfile(os.path.join(directory_in, file))]
file = list_file[0]

file

'crawler_1.xlsx'

In [4]:
print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(directory_in, file),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

df_input

Data Loading...
Loading Done !

总数量：54



,No,Url
0,1,https://toyota.oempartsonline.com/oem-parts/to...
1,2,https://toyota.oempartsonline.com/oem-parts/to...
2,3,https://toyota.oempartsonline.com/oem-parts/to...
3,4,https://toyota.oempartsonline.com/oem-parts/to...
4,5,https://toyota.oempartsonline.com/oem-parts/to...
5,6,https://toyota.oempartsonline.com/oem-parts/to...
6,7,https://toyota.oempartsonline.com/oem-parts/to...
7,8,https://toyota.oempartsonline.com/oem-parts/to...
8,9,https://toyota.oempartsonline.com/oem-parts/to...
9,10,https://toyota.oempartsonline.com/oem-parts/to...


In [5]:
crawler_status, (crawler_index, crawler_series) = False, next(df_input.iterrows())

crawler_status, crawler_index, crawler_series

(False,
 0,
 No                                                     1
 Url    https://toyota.oempartsonline.com/oem-parts/to...
 Name: 0, dtype: object)

In [6]:
import time

crawler_retry = 0
while True:
    crawler_retry += 1
    print(crawler_retry)
    try:
        url_request = crawler_series['Url']

        resp = requests.get(url_request,
                            headers=get_header(),
                            proxies=get_proxy(),
                            timeout=(5, 15))

        if resp.status_code == 200:
            soup = BeautifulSoup(resp.text, 'lxml')
            html = etree.HTML(str(soup))

            dict_ = json.loads(html.xpath('//script[@type="application/json"]/text()')[0])
            if dict_:
                with open(os.path.join(path_root, './part.html'), 'w', encoding='UTF-8') as file:
                    file.write(str(soup))
                break
    except requests.exceptions.ProxyError as e:
        if '456 CN Client Forbidden' in str(e):
            print('Long live the People\'s Republic of China!')
            break
    except KeyboardInterrupt:
        break
    except:
        time.sleep(3)
        continue

dict_

1


{'in_rp_catalog': True,
 'source': 'product',
 'sku': '23209-25070-01',
 'sku_stripped': '232092507001',
 'catalog': 'toyota',
 'price': 313.83,
 'msrp': 461.96,
 'images': [{'main': {'url': '//s3.amazonaws.com/static.revolutionparts.com/assets/images/toyota.png',
    'width': '',
    'height': ''},
   'preview': None,
   'thumb': None,
   'type': 'LOGO',
   'image_id': None,
   'alt_text': '',
   'is_primary': False,
   'is_visible': True,
   'filename': '',
   'has_watermark': 'Unknown',
   'md5': '',
   'perceptual_hash': '',
   'product_sku_stripped': None,
   'hotspot': {'reference_number': None},
   'caption': None}],
 'title': 'Fuel Injector - Toyota (23209-25070-01)',
 'fitment': [{'year': '2024',
   'make': 'Toyota',
   'model': 'Crown',
   'trims': ['Platinum'],
   'engines': ['2.4L L4 - Electric/Gas'],
   'idx': 0},
  {'year': '2024',
   'make': 'Toyota',
   'model': 'Grand Highlander',
   'trims': ['Hybrid Limited',
    'Hybrid Platinum',
    'Limited',
    'Platinum',
    

In [7]:
dict_src, list_img = {}, dict_['images']
for i, img in enumerate(list_img):
    dict_src[str(i)] = f'''https:{img['main']['url'].strip()}''' if img['main']['url'].strip().startswith('//') else img['main']['url'].strip()

dict_src

{'0': 'https://s3.amazonaws.com/static.revolutionparts.com/assets/images/toyota.png'}

In [8]:
list_dict_vehicle, list_dict_fitment = [], dict_['fitment']
for dict_fitment in list_dict_fitment:
    dict_fitment.pop('idx')
    list_key = list(dict_fitment.keys())
    list_value = [dict_fitment[key] if type(dict_fitment[key]) == list else [dict_fitment[key]] for key in list_key]

    product_combination = itertools.product(*list_value)
    for combination in product_combination:
        list_dict_vehicle.append(dict(zip(list_key, combination)))

list_dict_vehicle

[{'year': '2024',
  'make': 'Toyota',
  'model': 'Crown',
  'trims': 'Platinum',
  'engines': '2.4L L4 - Electric/Gas'},
 {'year': '2024',
  'make': 'Toyota',
  'model': 'Grand Highlander',
  'trims': 'Hybrid Limited',
  'engines': '2.4L L4 - Electric/Gas'},
 {'year': '2024',
  'make': 'Toyota',
  'model': 'Grand Highlander',
  'trims': 'Hybrid Limited',
  'engines': '2.4L L4 - Gas'},
 {'year': '2024',
  'make': 'Toyota',
  'model': 'Grand Highlander',
  'trims': 'Hybrid Platinum',
  'engines': '2.4L L4 - Electric/Gas'},
 {'year': '2024',
  'make': 'Toyota',
  'model': 'Grand Highlander',
  'trims': 'Hybrid Platinum',
  'engines': '2.4L L4 - Gas'},
 {'year': '2024',
  'make': 'Toyota',
  'model': 'Grand Highlander',
  'trims': 'Limited',
  'engines': '2.4L L4 - Electric/Gas'},
 {'year': '2024',
  'make': 'Toyota',
  'model': 'Grand Highlander',
  'trims': 'Limited',
  'engines': '2.4L L4 - Gas'},
 {'year': '2024',
  'make': 'Toyota',
  'model': 'Grand Highlander',
  'trims': 'Platinum'

In [9]:
subtitle = html.xpath('//div[@class="page-builder-layout-module desktop-only"]/descendant::p[@class="product-subtitle"]/text()')[0].strip() if html.xpath('//div[@class="page-builder-layout-module desktop-only"]/descendant::p[@class="product-subtitle"]/text()') else ''

subtitle

'2023-2024 Toyota - 232092507001'

In [10]:
dict_key, list_li = {}, html.xpath('//h2[@class="product-details-title"]/following-sibling::ul[@class="field-list"][1]/li')
for li in list_li:
    key = li.xpath('./span[@class="list-label"]')[0]
    key = etree.tostring(key).decode('utf-8')
    key = re.sub(r'<.*?>', '', key)
    key = re.sub(r' +', ' ', key)
    key = html_lib.unescape(key)
    key = key.strip().removesuffix(':').strip()
    if key == 'Brand':
        continue
    elif key in dict_key:
        dict_key[key] += 1
    else:
        dict_key[key] = 1

dict_detail, dict_count = {}, {}
for i, li in enumerate(list_li):
    key = li.xpath('./span[@class="list-label"]')[0]
    key = etree.tostring(key).decode('utf-8')
    key = re.sub(r'<.*?>', '', key)
    key = re.sub(r' +', ' ', key)
    key = html_lib.unescape(key)
    key = key.strip().removesuffix(':').strip()
    if key == 'Brand':
        list_src = li.xpath('./span[@class="list-value"]/div[@class="part-manufacturer"]/div[@class="manufacturer-logos"]/img/@src')
        list_src = ['.'.join(src.split('/')[-1].split('.')[:-1]).title().strip() for src in list_src]
        dict_detail[str(i)] = {key: ';'.join(list_src)}
    else:
        value = li.xpath('./span[contains(@class, "list-value")]')[0]
        value = etree.tostring(value).decode('utf-8')
        value = re.sub(r'<.*?>', '', value)
        value = re.sub(r' +', ' ', value)
        value = html_lib.unescape(value)
        if dict_key[key] == 1:
            dict_detail[str(i)] = {key: value.strip()}
        else:
            if key in dict_count:
                dict_count[key] += 1
            else:
                dict_count[key] = 1
            dict_detail[str(i)] = {f'{key}-{dict_count[key]}': value.strip()}

dict_detail

{'0': {'Brand': 'Toyota;Scion'},
 '1': {'SKU': '23209-25070-01'},
 '2': {'Other Names': 'Injector Set Fuel'},
 '3': {'Description': 'DI Injector. Mark 1. Incl.O-Rings,Retainer and Seals.'}}

In [11]:
dict_correct_temp = {'No': crawler_series['No'],
                     'Catalog': dict_['catalog'].title().strip(),
                     'Part Number': dict_['sku'].strip(),
                     'Title': dict_['title'].strip(),
                     'Subtitle': subtitle,
                     'Price': dict_['price'],
                     'Picture': '',
                     'Url': crawler_series['Url'],
                     'Json_Src': json.dumps(dict_src),
                     'Json_Detail': json.dumps(dict_detail),
                     'Json_CPN': f'''{dict_['catalog'].title().strip()};{dict_['sku'].strip()}'''}

dict_correct_temp

{'No': 1,
 'Catalog': 'Toyota',
 'Part Number': '23209-25070-01',
 'Title': 'Fuel Injector - Toyota (23209-25070-01)',
 'Subtitle': '2023-2024 Toyota - 232092507001',
 'Price': 313.83,
 'Picture': '',
 'Url': 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232092507001',
 'Json_Src': '{"0": "https://s3.amazonaws.com/static.revolutionparts.com/assets/images/toyota.png"}',
 'Json_Detail': '{"0": {"Brand": "Toyota;Scion"}, "1": {"SKU": "23209-25070-01"}, "2": {"Other Names": "Injector Set Fuel"}, "3": {"Description": "DI Injector. Mark 1. Incl.O-Rings,Retainer and Seals."}}',
 'Json_CPN': 'Toyota;23209-25070-01'}

In [12]:
list_dict_correct_temp = [{**dict_correct_temp, **dict_vehicle} for dict_vehicle in list_dict_vehicle]

list_dict_correct_temp

[{'No': 1,
  'Catalog': 'Toyota',
  'Part Number': '23209-25070-01',
  'Title': 'Fuel Injector - Toyota (23209-25070-01)',
  'Subtitle': '2023-2024 Toyota - 232092507001',
  'Price': 313.83,
  'Picture': '',
  'Url': 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232092507001',
  'Json_Src': '{"0": "https://s3.amazonaws.com/static.revolutionparts.com/assets/images/toyota.png"}',
  'Json_Detail': '{"0": {"Brand": "Toyota;Scion"}, "1": {"SKU": "23209-25070-01"}, "2": {"Other Names": "Injector Set Fuel"}, "3": {"Description": "DI Injector. Mark 1. Incl.O-Rings,Retainer and Seals."}}',
  'Json_CPN': 'Toyota;23209-25070-01',
  'year': '2024',
  'make': 'Toyota',
  'model': 'Crown',
  'trims': 'Platinum',
  'engines': '2.4L L4 - Electric/Gas'},
 {'No': 1,
  'Catalog': 'Toyota',
  'Part Number': '23209-25070-01',
  'Title': 'Fuel Injector - Toyota (23209-25070-01)',
  'Subtitle': '2023-2024 Toyota - 232092507001',
  'Price': 313.83,
  'Picture': '',
  'Url': 'https://toyota

In [13]:
list_dict_correct.extend(list_dict_correct_temp)

list_dict_correct

[{'No': 1,
  'Catalog': 'Toyota',
  'Part Number': '23209-25070-01',
  'Title': 'Fuel Injector - Toyota (23209-25070-01)',
  'Subtitle': '2023-2024 Toyota - 232092507001',
  'Price': 313.83,
  'Picture': '',
  'Url': 'https://toyota.oempartsonline.com/oem-parts/toyota-fuel-injector-232092507001',
  'Json_Src': '{"0": "https://s3.amazonaws.com/static.revolutionparts.com/assets/images/toyota.png"}',
  'Json_Detail': '{"0": {"Brand": "Toyota;Scion"}, "1": {"SKU": "23209-25070-01"}, "2": {"Other Names": "Injector Set Fuel"}, "3": {"Description": "DI Injector. Mark 1. Incl.O-Rings,Retainer and Seals."}}',
  'Json_CPN': 'Toyota;23209-25070-01',
  'year': '2024',
  'make': 'Toyota',
  'model': 'Crown',
  'trims': 'Platinum',
  'engines': '2.4L L4 - Electric/Gas'},
 {'No': 1,
  'Catalog': 'Toyota',
  'Part Number': '23209-25070-01',
  'Title': 'Fuel Injector - Toyota (23209-25070-01)',
  'Subtitle': '2023-2024 Toyota - 232092507001',
  'Price': 313.83,
  'Picture': '',
  'Url': 'https://toyota

In [14]:
crawler_status = True

crawler_status

True

In [15]:
crawler_series['Request_Url'] = url_request

list_series_error.append(crawler_series)

list_series_error

[No                                                             1
 Url            https://toyota.oempartsonline.com/oem-parts/to...
 Request_Url    https://toyota.oempartsonline.com/oem-parts/to...
 Name: 0, dtype: object]

In [16]:
print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct)
    df_correct['year'] = df_correct['year'].astype(int)
    df_correct = df_correct.sort_values(by=['No', 'Part Number', 'make', 'model', 'year', 'trims', 'engines'],
                                        ascending=[True, True, True, True, False, True, True],
                                        ignore_index=True)
    df_correct.to_excel(os.path.join(path_root, './test_part.xlsx'), index=False)

df_correct if list_dict_correct else ''

Data Outputting...


,No,Catalog,Part Number,Title,Subtitle,Price,Picture,Url,Json_Src,Json_Detail,Json_CPN,year,make,model,trims,engines
0,1,Toyota,23209-25070-01,Fuel Injector - Toyota (23209-25070-01),2023-2024 Toyota - 232092507001,313.83,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://s3.amazonaws.com/static.revolut...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;23209-25070-01,2024,Toyota,Crown,Platinum,2.4L L4 - Electric/Gas
1,1,Toyota,23209-25070-01,Fuel Injector - Toyota (23209-25070-01),2023-2024 Toyota - 232092507001,313.83,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://s3.amazonaws.com/static.revolut...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;23209-25070-01,2023,Toyota,Crown,Platinum,2.4L L4 - Electric/Gas
2,1,Toyota,23209-25070-01,Fuel Injector - Toyota (23209-25070-01),2023-2024 Toyota - 232092507001,313.83,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://s3.amazonaws.com/static.revolut...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;23209-25070-01,2024,Toyota,Grand Highlander,Hybrid Limited,2.4L L4 - Electric/Gas
3,1,Toyota,23209-25070-01,Fuel Injector - Toyota (23209-25070-01),2023-2024 Toyota - 232092507001,313.83,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://s3.amazonaws.com/static.revolut...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;23209-25070-01,2024,Toyota,Grand Highlander,Hybrid Limited,2.4L L4 - Gas
4,1,Toyota,23209-25070-01,Fuel Injector - Toyota (23209-25070-01),2023-2024 Toyota - 232092507001,313.83,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://s3.amazonaws.com/static.revolut...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;23209-25070-01,2024,Toyota,Grand Highlander,Hybrid Platinum,2.4L L4 - Electric/Gas
5,1,Toyota,23209-25070-01,Fuel Injector - Toyota (23209-25070-01),2023-2024 Toyota - 232092507001,313.83,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://s3.amazonaws.com/static.revolut...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;23209-25070-01,2024,Toyota,Grand Highlander,Hybrid Platinum,2.4L L4 - Gas
6,1,Toyota,23209-25070-01,Fuel Injector - Toyota (23209-25070-01),2023-2024 Toyota - 232092507001,313.83,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://s3.amazonaws.com/static.revolut...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;23209-25070-01,2024,Toyota,Grand Highlander,Limited,2.4L L4 - Electric/Gas
7,1,Toyota,23209-25070-01,Fuel Injector - Toyota (23209-25070-01),2023-2024 Toyota - 232092507001,313.83,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://s3.amazonaws.com/static.revolut...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;23209-25070-01,2024,Toyota,Grand Highlander,Limited,2.4L L4 - Gas
8,1,Toyota,23209-25070-01,Fuel Injector - Toyota (23209-25070-01),2023-2024 Toyota - 232092507001,313.83,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://s3.amazonaws.com/static.revolut...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;23209-25070-01,2024,Toyota,Grand Highlander,Platinum,2.4L L4 - Electric/Gas
9,1,Toyota,23209-25070-01,Fuel Injector - Toyota (23209-25070-01),2023-2024 Toyota - 232092507001,313.83,,https://toyota.oempartsonline.com/oem-parts/to...,"{""0"": ""https://s3.amazonaws.com/static.revolut...","{""0"": {""Brand"": ""Toyota;Scion""}, ""1"": {""SKU"": ...",Toyota;23209-25070-01,2024,Toyota,Grand Highlander,Platinum,2.4L L4 - Gas


In [17]:
print('Output Finished !')
print()

Output Finished !



In [18]:
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(os.path.join(path_root, './test_part_error.xlsx'), index=False)
    print('- Error Occurred -')
    print()

df_error if list_series_error else ''

- Error Occurred -



,No,Url,Request_Url
0,1,https://toyota.oempartsonline.com/oem-parts/to...,https://toyota.oempartsonline.com/oem-parts/to...


In [19]:
print('Done ~')

Done ~
